In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/Users/renee/Downloads/Quarto Template (2023 REU)':
  os.chdir(r'/Users/renee/Downloads/Quarto Template (2023 REU)')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


In [2]:
# | eval: true

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import pickle

In [3]:
data = pd.read_csv('./data/iris.csv')
X = data.iloc[:, 0:4]
y = data.Species
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

clf = RandomForestClassifier(max_depth=2, random_state=0, oob_score=True)
clf.fit(X, y)

with open('./pickle_data/example_rf.pkl', 'wb') as out:
  pickle.dump(clf, out, pickle.HIGHEST_PROTOCOL)

In [4]:
#| eval: true

import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
with open('./pickle_data/example_rf.pkl', 'rb') as inp:
  clf = pickle.load(inp)

res = pd.DataFrame({'Setosa': clf.oob_decision_function_[:, 0], 
                    'Versicolor': clf.oob_decision_function_[:, 1],
                    'Virginica': clf.oob_decision_function_[:, 2]})

sns.pairplot(res)
plt.savefig('./figures/rf_res.pdf', format='pdf')
plt.savefig('./figures/rf_res.png', format='png')

In [6]:
#| eval: true
#| echo: false
import session_info
session_info.show()